# Notebook 01: Algoritmos Interactivos - Pensamiento Estructurado

**Clase Extra de Python - AI Engineering**

---

## Objetivos de Aprendizaje

Al finalizar este notebook seras capaz de:

1. **Resolver problemas** utilizando las tecnicas mas comunes en entrevistas tecnicas:
   - Hash Maps (diccionarios)
   - Two Pointers (dos punteros)
   - Sliding Window (ventana deslizante)
   - Dynamic Programming (programacion dinamica)
   - Binary Search (busqueda binaria)
2. **Analizar la complejidad** temporal y espacial de cada solucion.
3. **Identificar patrones reutilizables** que se aplican a decenas de problemas distintos.

---

**Tiempo estimado:** 90 - 120 minutos

**Problemas cubiertos:**

| # | Problema | Patron | Dificultad LeetCode |
|---|----------|--------|---------------------|
| 1 | Two Sum | Hash Map | Easy |
| 2 | Valid Palindrome | Two Pointers | Easy |
| 3 | Climbing Stairs | Dynamic Programming | Easy |
| 4 | Binary Search | Binary Search | Easy |
| 5 | Group Anagrams | Hash Map | Medium |
| 6 | Longest Substring Without Repeating Characters | Sliding Window | Medium |

**Requisitos:** Solo biblioteca estandar de Python. No se necesitan dependencias externas.

In [ ]:
# =============================================================================
# SETUP - Importaciones y utilidades
# =============================================================================

from typing import List, Optional, Dict
import collections
import time


def verify(test_name: str, got, expected) -> None:
    """Funcion auxiliar para verificar resultados de tests.
    
    Compara el valor obtenido con el esperado e imprime PASS o FAIL.
    """
    status = "PASS" if got == expected else "FAIL"
    print(f"  [{status}] {test_name}: got={got}, expected={expected}")


print("Setup completo. Todas las utilidades cargadas.")

---

## Problema 1: Two Sum

**LeetCode #1 - Easy**

### Enunciado

Dado un arreglo de enteros `nums` y un entero `target`, retorna los **indices** de los dos
numeros que suman `target`.

Puedes asumir que cada entrada tiene **exactamente una solucion** y no puedes usar el
mismo elemento dos veces.

### Ejemplos

```
Input:  nums = [2, 7, 11, 15], target = 9
Output: [0, 1]   # porque nums[0] + nums[1] = 2 + 7 = 9

Input:  nums = [3, 2, 4], target = 6
Output: [1, 2]   # porque nums[1] + nums[2] = 2 + 4 = 6
```

### Pista: Hash Map

En lugar de probar todos los pares (fuerza bruta O(n^2)), podemos usar un diccionario
para recordar los numeros que ya vimos:

```
nums = [2, 7, 11, 15]   target = 9

Paso 1: n=2  ->  need=7  ->  seen={}         -> 7 no esta  -> seen={2:0}
Paso 2: n=7  ->  need=2  ->  seen={2:0}      -> 2 SI esta! -> return [0, 1]
         ^                         ^
         |                         |
    indice actual         indice guardado de 2
```

La clave es: para cada numero `n`, calculamos `need = target - n` y preguntamos
si `need` ya lo vimos antes.

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Two Sum
# =============================================================================

def two_sum(nums: List[int], target: int) -> Optional[List[int]]:
    """
    Encuentra dos indices cuyos valores suman target.
    
    Invariante: 'seen' contiene todos los numeros vistos hasta ahora
    junto con sus indices. Si el complemento (target - n) esta en 'seen',
    encontramos la respuesta.
    
    Complejidad: O(n) tiempo, O(n) espacio.
    """
    seen: Dict[int, int] = {}  # valor -> indice
    
    for i, n in enumerate(nums):
        need = target - n          # el complemento que necesitamos
        
        if need in seen:           # ya vimos el complemento?
            return [seen[need], i] # retornamos ambos indices
        
        seen[n] = i                # guardamos el numero actual
    
    return None  # no se encontro solucion


# --- Tests ---
print("Two Sum - Tests:")
verify("ejemplo basico",      two_sum([2, 7, 11, 15], 9),  [0, 1])
verify("numeros no contiguos", two_sum([3, 2, 4], 6),       [1, 2])
verify("numeros repetidos",    two_sum([3, 3], 6),           [0, 1])
verify("sin solucion",         two_sum([1, 2], 10),          None)

### Analisis de Complejidad - Two Sum

| Enfoque | Tiempo | Espacio | Explicacion |
|---------|--------|---------|-------------|
| Fuerza bruta (dos loops) | O(n^2) | O(1) | Probamos todos los pares posibles |
| **Hash Map (nuestra solucion)** | **O(n)** | **O(n)** | Un solo recorrido; lookup en O(1) |

**Trade-off clave:** Invertimos **espacio extra O(n)** para ganar **velocidad O(n)** en vez de O(n^2).

Este es uno de los trade-offs mas fundamentales en algoritmia:
**usar memoria adicional para reducir tiempo de ejecucion.**

Para n = 10,000:
- Fuerza bruta: ~100,000,000 operaciones
- Hash Map: ~10,000 operaciones (10,000x mas rapido)

---

## Problema 2: Valid Palindrome

**LeetCode #125 - Easy**

### Enunciado

Una frase es un **palindromo** si, despues de convertir todas las letras a minusculas
y remover todos los caracteres no alfanumericos, se lee igual de izquierda a derecha
que de derecha a izquierda.

### Ejemplos

```
Input:  "A man, a plan, a canal: Panama"
Output: True   # "amanaplanacanalpanama" es palindromo

Input:  "race a car"
Output: False  # "raceacar" no es palindromo
```

### Pista: Two Pointers (Dos Punteros)

Usamos un puntero al inicio y otro al final. Los movemos hacia el centro
saltando caracteres no alfanumericos:

```
"A man, a plan, a canal: Panama"
 ^                            ^
 i                            j
 A == a ? Si -> avanzar ambos

"A man, a plan, a canal: Panama"
  ^                          ^
  i                          j
  m == m ? Si -> avanzar ambos

  ... (saltar espacios, comas, dos puntos) ...

  Cuando i >= j -> es palindromo!
```

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Valid Palindrome
# =============================================================================

def is_palindrome(s: str) -> bool:
    """
    Verifica si una cadena es palindromo (ignorando caracteres no alfanumericos
    y sin distinguir mayusculas/minusculas).
    
    Tecnica: Two Pointers - un puntero al inicio, otro al final.
    
    Complejidad: O(n) tiempo, O(1) espacio.
    """
    i, j = 0, len(s) - 1
    
    while i < j:
        # Saltar caracteres no alfanumericos desde la izquierda
        while i < j and not s[i].isalnum():
            i += 1
        
        # Saltar caracteres no alfanumericos desde la derecha
        while i < j and not s[j].isalnum():
            j -= 1
        
        # Comparar (case-insensitive)
        if s[i].lower() != s[j].lower():
            return False
        
        i += 1
        j -= 1
    
    return True


# --- Tests ---
print("Valid Palindrome - Tests:")
verify("palindromo clasico",   is_palindrome("A man, a plan, a canal: Panama"), True)
verify("no palindromo",        is_palindrome("race a car"),                      False)
verify("cadena vacia/espacio", is_palindrome(" "),                               True)
verify("un solo caracter",     is_palindrome("a"),                               True)
verify("solo simbolos",        is_palindrome(".,"),                              True)

### Analisis de Complejidad - Valid Palindrome

| Aspecto | Valor |
|---------|-------|
| Tiempo  | O(n) - recorremos la cadena una sola vez |
| Espacio | O(1) - solo usamos dos variables (i, j) |

**Comparacion con enfoque alternativo:**

Podriamos crear una nueva cadena filtrada y compararla con su reverso:
```python
filtered = ''.join(c.lower() for c in s if c.isalnum())
return filtered == filtered[::-1]
```
Esto tambien es O(n) en tiempo, pero **O(n) en espacio** porque creamos una cadena nueva.
La solucion con dos punteros es mas eficiente en memoria.

---

## Problema 3: Climbing Stairs (Dynamic Programming)

**LeetCode #70 - Easy**

### Enunciado

Estas subiendo una escalera de `n` peldanos. En cada paso puedes subir **1 o 2 peldanos**.
De cuantas formas distintas puedes llegar a la cima?

### Ejemplos

```
Input:  n = 2
Output: 2   # (1+1) o (2)

Input:  n = 3
Output: 3   # (1+1+1), (1+2), (2+1)
```

### Visualizacion: Arbol de Decisiones

Para n=4, el arbol de decisiones se ve asi:

```
                        f(4)
                       /    \
                    f(3)    f(2)        <- subproblemas solapados!
                   /    \   /    \
                f(2)  f(1) f(1) f(0)
               /    \
            f(1)   f(0)
```

Notamos que `f(2)` se calcula multiples veces. Esto es exactamente lo que
la Programacion Dinamica resuelve.

### Relacion de Recurrencia

```
f(n) = f(n-1) + f(n-2)
```

Esto es identico a la **secuencia de Fibonacci**:

```
n:    1  2  3  4  5  6  7   8
f(n): 1  2  3  5  8  13 21  34
```

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Climbing Stairs
# =============================================================================

def climb_stairs(n: int) -> int:
    """
    Calcula el numero de formas distintas de subir n peldanos,
    pudiendo subir 1 o 2 peldanos a la vez.
    
    Recurrencia: f(n) = f(n-1) + f(n-2)
    Caso base:   f(1) = 1, f(2) = 2
    
    Usamos solo dos variables en vez de un arreglo completo
    (optimizacion de espacio).
    
    Complejidad: O(n) tiempo, O(1) espacio.
    """
    if n <= 2:
        return n
    
    # a = f(n-2), b = f(n-1)
    a, b = 1, 2
    
    for _ in range(3, n + 1):
        # f(n) = f(n-1) + f(n-2)
        a, b = b, a + b
    
    return b


# --- Tests ---
print("Climbing Stairs - Tests:")
verify("n=1", climb_stairs(1), 1)
verify("n=2", climb_stairs(2), 2)
verify("n=3", climb_stairs(3), 3)
verify("n=4", climb_stairs(4), 5)
verify("n=5", climb_stairs(5), 8)

# Mostrar la relacion con Fibonacci
print("\nSecuencia de Climbing Stairs (es Fibonacci desplazado):")
for i in range(1, 11):
    print(f"  n={i:2d} -> {climb_stairs(i)} formas")

### Analisis de Complejidad - Climbing Stairs

| Enfoque | Tiempo | Espacio | Explicacion |
|---------|--------|---------|-------------|
| Recursion pura | O(2^n) | O(n) stack | Recalcula subproblemas multiples veces |
| Memoizacion (top-down) | O(n) | O(n) | Guarda resultados en diccionario/arreglo |
| **Tabulacion optimizada** | **O(n)** | **O(1)** | Solo necesitamos los 2 valores anteriores |

**Leccion clave de DP:**
1. Identificar la **relacion de recurrencia**: f(n) = f(n-1) + f(n-2)
2. Identificar los **casos base**: f(1)=1, f(2)=2
3. Decidir la **direccion** (top-down vs bottom-up)
4. **Optimizar espacio** si solo necesitamos los ultimos k valores

---

## Problema 4: Binary Search

**LeetCode #704 - Easy**

### Enunciado

Dado un arreglo **ordenado** de enteros `nums` y un entero `target`, implementa
la busqueda binaria. Retorna el indice de `target` si existe, o `-1` si no.

### Ejemplos

```
Input:  nums = [-1, 0, 3, 5, 9, 12], target = 9
Output: 4

Input:  nums = [-1, 0, 3, 5, 9, 12], target = 2
Output: -1
```

### Visualizacion: Division del Espacio de Busqueda

```
Buscando target=9 en [-1, 0, 3, 5, 9, 12]

Paso 1:  [-1, 0, 3, 5, 9, 12]
          lo        mid       hi
          0         2         5
          nums[2]=3 < 9  ->  lo = mid + 1 = 3

Paso 2:  [-1, 0, 3, 5, 9, 12]
                     lo  mid  hi
                     3   4    5
          nums[4]=9 == 9  ->  return 4
```

### Invariante

**Si el target existe en el arreglo, esta en `nums[lo..hi]` (inclusive).**

Cada iteracion reduce el espacio de busqueda a la mitad.

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Binary Search
# =============================================================================

def binary_search(nums: List[int], target: int) -> int:
    """
    Busqueda binaria en un arreglo ordenado.
    
    Invariante: si target existe, esta en nums[lo..hi] (inclusive).
    En cada paso, descartamos la mitad del espacio de busqueda.
    
    Complejidad: O(log n) tiempo, O(1) espacio.
    """
    lo, hi = 0, len(nums) - 1
    
    while lo <= hi:
        mid = (lo + hi) // 2       # punto medio
        
        if nums[mid] == target:    # encontrado!
            return mid
        elif nums[mid] < target:   # target esta a la derecha
            lo = mid + 1
        else:                      # target esta a la izquierda
            hi = mid - 1
    
    return -1  # no encontrado


# --- Tests ---
print("Binary Search - Tests:")
verify("target existe",        binary_search([-1, 0, 3, 5, 9, 12], 9),  4)
verify("target no existe",     binary_search([-1, 0, 3, 5, 9, 12], 2),  -1)
verify("arreglo de 1",         binary_search([5], 5),                    0)
verify("arreglo vacio",        binary_search([], 1),                     -1)
verify("primer elemento",      binary_search([1, 3, 5, 7], 1),          0)
verify("ultimo elemento",      binary_search([1, 3, 5, 7], 7),          3)

# Demostrar la eficiencia logaritmica
print("\nEficiencia logaritmica:")
for size in [10, 100, 1_000, 1_000_000, 1_000_000_000]:
    import math
    pasos = math.ceil(math.log2(size + 1))
    print(f"  n={size:>13,} -> maximo ~{pasos:>2} comparaciones")

---

## Problema 5: Group Anagrams

**LeetCode #49 - Medium**

### Enunciado

Dado un arreglo de cadenas `strs`, agrupa los **anagramas** juntos. Puedes retornar
los grupos en cualquier orden.

Un **anagrama** es una palabra formada reordenando las letras de otra palabra,
usando todas las letras originales exactamente una vez.

### Ejemplos

```
Input:  ["eat", "tea", "tan", "ate", "nat", "bat"]
Output: [["eat","tea","ate"], ["tan","nat"], ["bat"]]
```

### Concepto Clave: Firma/Llave

Dos palabras son anagramas si y solo si tienen la **misma frecuencia de caracteres**.
Podemos usar esta frecuencia como llave de un diccionario:

```
"eat" -> conteo: a=1, e=1, t=1 -> key = (1,0,0,0,1,0,...,1,0,...)
"tea" -> conteo: a=1, e=1, t=1 -> key = (1,0,0,0,1,0,...,1,0,...)
"tan" -> conteo: a=1, n=1, t=1 -> key = (1,0,0,0,0,...,1,...,1,...)
                                              ^
                      "eat" y "tea" comparten la misma llave!
```

Usamos un arreglo de 26 posiciones (una por letra) como llave.

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Group Anagrams
# =============================================================================

def group_anagrams(strs: List[str]) -> List[List[str]]:
    """
    Agrupa strings que son anagramas entre si.
    
    Estrategia: usamos el conteo de caracteres como llave del diccionario.
    Dos palabras son anagramas sii tienen el mismo conteo de caracteres.
    
    Complejidad: O(n * k) tiempo, donde n = len(strs), k = max longitud de string.
                 O(n * k) espacio para almacenar los grupos.
    """
    groups: Dict[tuple, List[str]] = {}
    
    for s in strs:
        # Construir la llave: conteo de frecuencia de cada letra
        count = [0] * 26
        for ch in s:
            count[ord(ch) - ord('a')] += 1
        
        key = tuple(count)  # las listas no son hashables, las tuplas si
        
        # Agrupar por llave
        groups.setdefault(key, []).append(s)
    
    return list(groups.values())


# --- Tests ---
print("Group Anagrams - Tests:")

# Para comparar grupos sin importar el orden, ordenamos internamente
def sort_groups(groups: List[List[str]]) -> List[List[str]]:
    """Ordena cada grupo y luego ordena la lista de grupos para comparacion."""
    return sorted([sorted(g) for g in groups])

result1 = group_anagrams(["eat", "tea", "tan", "ate", "nat", "bat"])
expected1 = [["eat", "tea", "ate"], ["tan", "nat"], ["bat"]]
verify("ejemplo clasico",
       sort_groups(result1),
       sort_groups(expected1))

result2 = group_anagrams([""])
verify("cadena vacia", result2, [[""]])

result3 = group_anagrams(["a"])
verify("un solo elemento", result3, [["a"]])

result4 = group_anagrams(["abc", "def", "ghi"])
verify("sin anagramas",
       sort_groups(result4),
       sort_groups([["abc"], ["def"], ["ghi"]]))

### Analisis de Complejidad - Group Anagrams

| Enfoque | Tiempo | Espacio |
|---------|--------|---------|  
| Ordenar cada string como llave | O(n * k * log k) | O(n * k) |
| **Conteo de caracteres como llave** | **O(n * k)** | **O(n * k)** |

Donde `n` = cantidad de strings y `k` = longitud maxima de un string.

**Por que conteo es mejor que sorting?** Porque contar caracteres es O(k) mientras
que ordenar una cadena es O(k log k). Para strings largos, la diferencia importa.

---

## Problema 6: Longest Substring Without Repeating Characters (Sliding Window)

**LeetCode #3 - Medium**

### Enunciado

Dada una cadena `s`, encuentra la longitud de la **subcadena mas larga sin caracteres
repetidos**.

### Ejemplos

```
Input:  "abcabcbb"
Output: 3   # "abc"

Input:  "bbbbb"
Output: 1   # "b"

Input:  "pwwkew"
Output: 3   # "wke"
```

### Pista: Ventana Deslizante (Sliding Window)

Mantenemos una ventana `[left, right]` donde no hay caracteres repetidos.
Cuando encontramos un caracter repetido, movemos `left` hacia adelante:

```
s = "abcabcbb"

Paso 1:  [a] b c a b c b b    left=0, right=0, best=1
Paso 2:  [a b] c a b c b b    left=0, right=1, best=2
Paso 3:  [a b c] a b c b b    left=0, right=2, best=3
Paso 4:   a [b c a] b c b b   left=1, right=3, best=3  <- 'a' repetida, mover left
Paso 5:   a b [c a b] c b b   left=2, right=4, best=3
Paso 6:   a b c [a b c] b b   left=3, right=5, best=3
Paso 7:   a b c a b [c b] b   left=5, right=6, best=3  <- 'b' repetida, mover left
Paso 8:   a b c a b c [b] b   left=6, right=7, best=3  <- 'b' repetida, mover left
                                                  
Resultado: 3
```

In [ ]:
# =============================================================================
# SOLUCION GUIADA: Longest Substring Without Repeating Characters
# =============================================================================

def length_of_longest_substring(s: str) -> int:
    """
    Encuentra la longitud de la subcadena mas larga sin caracteres repetidos.
    
    Tecnica: Sliding Window con diccionario.
    - 'last' guarda la ultima posicion donde vimos cada caracter.
    - 'left' es el inicio de la ventana actual.
    - Cuando encontramos un caracter repetido dentro de la ventana,
      movemos 'left' justo despues de la ultima aparicion.
    
    Complejidad: O(n) tiempo, O(min(n, m)) espacio donde m = tamano del alfabeto.
    """
    last: Dict[str, int] = {}  # caracter -> ultima posicion vista
    left = 0                    # inicio de la ventana
    best = 0                    # mejor longitud encontrada
    
    for right, ch in enumerate(s):
        # Si el caracter ya fue visto Y esta dentro de la ventana actual
        if ch in last and last[ch] >= left:
            # Mover el inicio justo despues de la aparicion anterior
            left = last[ch] + 1
        
        # Actualizar la ultima posicion del caracter
        last[ch] = right
        
        # Actualizar el mejor resultado
        best = max(best, right - left + 1)
    
    return best


# --- Tests ---
print("Longest Substring Without Repeating Characters - Tests:")
verify("ejemplo 1",       length_of_longest_substring("abcabcbb"), 3)
verify("todos iguales",   length_of_longest_substring("bbbbb"),    1)
verify("ejemplo 3",       length_of_longest_substring("pwwkew"),   3)
verify("cadena vacia",    length_of_longest_substring(""),         0)
verify("sin repetidos",   length_of_longest_substring("abcdef"),   6)
verify("un caracter",     length_of_longest_substring("a"),        1)

### Analisis de Complejidad - Longest Substring

| Enfoque | Tiempo | Espacio |
|---------|--------|---------|
| Fuerza bruta (todas las subcadenas) | O(n^3) | O(n) |
| Fuerza bruta optimizada | O(n^2) | O(n) |
| **Sliding Window** | **O(n)** | **O(min(n, m))** |

Donde `m` = tamano del alfabeto (26 para minusculas, 128 para ASCII).

**Patron Sliding Window - Cuando usarlo:**
- Buscamos una **subcadena o subarreglo** con cierta propiedad.
- La propiedad se puede mantener/actualizar incrementalmente.
- Al expandir la ventana, a veces necesitamos contraerla.

---

## Resumen de Patrones

Estos 6 problemas cubren los 5 patrones mas importantes en entrevistas tecnicas:

| Patron | Problemas | Cuando usarlo | Complejidad tipica |
|--------|-----------|---------------|--------------------|
| **Hash Map** | Two Sum, Group Anagrams | Necesitas lookup rapido O(1), contar frecuencias, agrupar elementos | O(n) tiempo, O(n) espacio |
| **Two Pointers** | Valid Palindrome | Datos ordenados o simetricos, buscar pares | O(n) tiempo, O(1) espacio |
| **Sliding Window** | Longest Substring | Subcadena/subarreglo con propiedad, optimizar rango contiguo | O(n) tiempo, O(k) espacio |
| **Dynamic Programming** | Climbing Stairs | Subproblemas solapados, decision optima, conteo de caminos | O(n) tiempo, O(1)-O(n) espacio |
| **Binary Search** | Binary Search | Datos ordenados, buscar umbral, minimizar/maximizar | O(log n) tiempo, O(1) espacio |

### Guia rapida para elegir patron

```
El problema pide...                     ->  Patron probable
------------------------------------------------------------
Encontrar par/grupo con propiedad       ->  Hash Map
Subcadena/subarreglo optimo             ->  Sliding Window
Comparar extremos de secuencia          ->  Two Pointers
Contar caminos / optimo con elecciones  ->  Dynamic Programming
Buscar en datos ordenados               ->  Binary Search
```

---

## Ejercicios para Practicar

Ahora es tu turno. Intenta resolver los siguientes problemas aplicando los patrones
que acabamos de aprender.

In [ ]:
# =============================================================================
# EJERCICIO 1: Three Sum (LeetCode #15 - Medium)
# =============================================================================
#
# Dado un arreglo de enteros 'nums', encuentra todos los tripletes unicos
# [nums[i], nums[j], nums[k]] tales que i != j != k y
# nums[i] + nums[j] + nums[k] == 0.
#
# Ejemplo:
#   Input:  nums = [-1, 0, 1, 2, -1, -4]
#   Output: [[-1, -1, 2], [-1, 0, 1]]
#
# Pista: Ordena el arreglo primero. Luego, para cada elemento nums[i],
#        usa Two Pointers para buscar un par que sume -nums[i] en el
#        subarreglo restante. Cuidado con los duplicados!
#
# Complejidad esperada: O(n^2) tiempo, O(1) espacio extra (sin contar output)

def three_sum(nums: List[int]) -> List[List[int]]:
    """Encuentra todos los tripletes que suman 0."""
    pass  # <-- Tu implementacion aqui


# --- Tests (descomenta cuando tengas tu solucion) ---
# print("Three Sum - Tests:")
# result = three_sum([-1, 0, 1, 2, -1, -4])
# expected = [[-1, -1, 2], [-1, 0, 1]]
# verify("ejemplo clasico", sorted([sorted(t) for t in result]), sorted([sorted(t) for t in expected]))
# verify("sin solucion", three_sum([0, 1, 1]), [])
# verify("todos ceros", three_sum([0, 0, 0]), [[0, 0, 0]])

In [ ]:
# =============================================================================
# EJERCICIO 2: Valid Anagram (LeetCode #242 - Easy)
# =============================================================================
#
# Dados dos strings s y t, determina si t es un anagrama de s.
#
# Ejemplo:
#   Input:  s = "anagram", t = "nagaram"
#   Output: True
#
#   Input:  s = "rat", t = "car"
#   Output: False
#
# Dos enfoques posibles:
#   1. Sorting: ordenar ambas cadenas y comparar -> O(n log n)
#   2. Counting: contar frecuencia de cada caracter -> O(n)
#
# Implementa AMBOS enfoques y compara:

def is_anagram_sort(s: str, t: str) -> bool:
    """Verifica anagrama usando sorting."""
    pass  # <-- Tu implementacion aqui


def is_anagram_count(s: str, t: str) -> bool:
    """Verifica anagrama usando conteo de caracteres."""
    pass  # <-- Tu implementacion aqui


# --- Tests (descomenta cuando tengas tu solucion) ---
# print("Valid Anagram - Tests (sorting):")
# verify("anagrama valido",   is_anagram_sort("anagram", "nagaram"), True)
# verify("no es anagrama",    is_anagram_sort("rat", "car"),         False)
# verify("distinta longitud", is_anagram_sort("ab", "abc"),          False)
#
# print("\nValid Anagram - Tests (counting):")
# verify("anagrama valido",   is_anagram_count("anagram", "nagaram"), True)
# verify("no es anagrama",    is_anagram_count("rat", "car"),         False)
# verify("distinta longitud", is_anagram_count("ab", "abc"),          False)

In [ ]:
# =============================================================================
# EJERCICIO 3: Maximum Subarray (LeetCode #53 - Medium)
# =============================================================================
#
# Dado un arreglo de enteros 'nums', encuentra el subarreglo contiguo
# con la suma mas grande y retorna su suma.
#
# Ejemplo:
#   Input:  nums = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
#   Output: 6   # El subarreglo [4, -1, 2, 1] tiene la suma mas grande = 6
#
# Algoritmo de Kadane:
#   - Recorre el arreglo manteniendo la suma actual (current_sum)
#   - Si current_sum se vuelve negativa, resetea a 0
#     (mejor empezar de nuevo que arrastrar una suma negativa)
#   - Guarda el maximo visto (max_sum)
#
# Complejidad esperada: O(n) tiempo, O(1) espacio

def max_subarray(nums: List[int]) -> int:
    """Encuentra la suma maxima de un subarreglo contiguo (Kadane's algorithm)."""
    pass  # <-- Tu implementacion aqui


# --- Tests (descomenta cuando tengas tu solucion) ---
# print("Maximum Subarray - Tests:")
# verify("ejemplo clasico",    max_subarray([-2, 1, -3, 4, -1, 2, 1, -5, 4]), 6)
# verify("un solo elemento",   max_subarray([1]),                               1)
# verify("todos negativos",    max_subarray([-1, -2, -3]),                      -1)
# verify("todos positivos",    max_subarray([1, 2, 3, 4]),                      10)

---

## Checklist de Consolidacion

Antes de terminar, verifica que puedas responder estas preguntas:

- [ ] **Hash Map:** Puedo explicar por que Two Sum pasa de O(n^2) a O(n) con un diccionario.
- [ ] **Two Pointers:** Puedo implementar la tecnica de dos punteros para recorrer una secuencia desde ambos extremos.
- [ ] **Sliding Window:** Puedo mantener una ventana deslizante y ajustar sus bordes segun una condicion.
- [ ] **Dynamic Programming:** Puedo identificar subproblemas solapados y escribir la relacion de recurrencia.
- [ ] **Binary Search:** Puedo implementar busqueda binaria sin errores de off-by-one (lo <= hi, mid+1, mid-1).
- [ ] **Analisis de complejidad:** Para cada solucion, puedo justificar su complejidad temporal y espacial.
- [ ] **Reconocimiento de patrones:** Dado un problema nuevo, puedo identificar cual patron aplicar.
- [ ] **Trade-offs:** Puedo explicar cuando conviene usar mas memoria para ganar velocidad.

---

### Recursos Adicionales

- [NeetCode 150](https://neetcode.io/practice) - Problemas organizados por patron
- [Blind 75](https://leetcode.com/discuss/general-discussion/460599/blind-75-leetcode-questions) - Los 75 problemas mas frecuentes en entrevistas
- [Big-O Cheat Sheet](https://www.bigocheatsheet.com/) - Referencia rapida de complejidades

---

*Notebook 01 - Algoritmos Interactivos | Clase Extra de Python - AI Engineering*